## Sétima Tentativa

Tendo em vista a análise de tentativas anteriores, nessa, haverá um plano de ação específico. Os estimadores de foco utilizados serão o ``VotingClassifier`` e o ``StackingClassifier`` fazendo uso de modelos com performance já registrada anteriormente, a Rede Neural Otimizada da 5ª Tentativa e novos modelos ainda não tentados.

Fora isso, eu também desejo investigar a possibilidade do uso de *Feature Selection* novamente, com algoritmos novos.

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/santander-customer-satisfaction/sample_submission.csv
/kaggle/input/santander-customer-satisfaction/train.csv
/kaggle/input/santander-customer-satisfaction/test.csv


### 1. Preparações
Dessa vez, vamos criar um conjunto com as colunas constantes, para avaliar os resultados.

In [2]:
dataset = pd.read_csv("/kaggle/input/santander-customer-satisfaction/train.csv")

In [3]:
ID = dataset.ID
TARGET = dataset.TARGET

dataset.drop(columns = "ID", inplace = True)
dataset.drop(columns = "TARGET", inplace = True)

In [4]:
COLUNAS_CONSTANTES = [coluna for coluna in dataset.columns if dataset[coluna].nunique() == 1]
dataset_modificado = dataset.drop(columns = COLUNAS_CONSTANTES)
dataset_modificado.shape

(76020, 335)

In [5]:
dataset.shape

(76020, 369)

#### 1.1 Escalando o Conjunto

In [6]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

escalado = scaler.fit_transform(dataset.copy())

dataset = pd.DataFrame(escalado, columns = dataset.columns)
dataset.head()

,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,imp_op_var40_ult1,...,saldo_medio_var29_ult3,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38
0,0.999764,0.18,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.001545
1,0.999764,0.29,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.002003
2,0.999764,0.18,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.002822
3,0.999764,0.32,0.0,0.01513,0.009275,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.002671
4,0.999764,0.34,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.005091


In [7]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

escalado = scaler.fit_transform(dataset_modificado.copy())

dataset_modificado = pd.DataFrame(escalado, columns = dataset_modificado.columns)
dataset_modificado.head()

,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,imp_op_var40_ult1,...,saldo_medio_var29_ult3,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38
0,0.999764,0.18,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.001545
1,0.999764,0.29,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.002003
2,0.999764,0.18,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.002822
3,0.999764,0.32,0.0,0.01513,0.009275,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.002671
4,0.999764,0.34,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.005091


#### 1.2 Undersampling
O Oversampling já se provou, de novo e de novo, a apresentar menor performance. Assim, vamos abandonar a abordagem por completo aqui e trabalhar somente com o Undersampling

In [8]:
from imblearn.under_sampling import RandomUnderSampler

SEED = 912039

In [9]:
sampler = RandomUnderSampler(random_state = SEED)
x_under, y_under = sampler.fit_resample(dataset, TARGET)
dataset = pd.concat([x_under, y_under], axis = 1)
dataset.TARGET.value_counts()

TARGET
0    3008
1    3008
Name: count, dtype: int64

In [10]:
sampler = RandomUnderSampler(random_state = SEED)
x_under, y_under = sampler.fit_resample(dataset_modificado, TARGET)
dataset_modificado = pd.concat([x_under, y_under], axis = 1)
dataset_modificado.TARGET.value_counts()

TARGET
0    3008
1    3008
Name: count, dtype: int64

In [11]:
dataset.TARGET.value_counts() == dataset_modificado.TARGET.value_counts()

TARGET
0    True
1    True
Name: count, dtype: bool

In [12]:
y = dataset.TARGET

dataset.drop("TARGET", axis = 1, inplace = True)
dataset_modificado.drop("TARGET", axis = 1, inplace = True)

### 2. Modelos
Simplesmente adicionar modelos a torto e a direito ao nossos ensemble learners não é necessariamente uma boa prática. Vamos fazer uma avaliação individual de cada modelo primeiro, para depois testá-los de maneira conjunta.

In [13]:
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer, roc_auc_score

def avalia_modelo(modelo, x, y, cv = 10):
    scoring = {
        'accuracy': 'accuracy',
        'precision': 'precision',
        'recall': 'recall',
        'f1': 'f1',
        'roc_auc': make_scorer(roc_auc_score)
    }
    
    resultados = cross_validate(modelo, x, y, scoring=scoring, cv=cv, n_jobs=-1)
    
    print(f'Modelo: {modelo}')
    media_acuracia = sum(resultados['test_accuracy'])/cv
    media_precisao = sum(resultados['test_precision'])/cv
    media_recall = sum(resultados['test_recall'])/cv
    media_f1 = sum(resultados['test_f1'])/cv
    media_roc_auc = sum(resultados['test_roc_auc'])/cv
    
    print(f'Acurácia --> {media_acuracia :.2f}')
    print(f'Precisão --> {media_precisao :.2f}')
    print(f'Recall --> {media_recall :.2f}')
    print(f'F1 --> {media_f1 :.2f}')
    print(f'ROC AUC --> {media_roc_auc :.2f}')

In [14]:
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.neural_network import MLPClassifier
import xgboost as xgb

svm = SVC(random_state = SEED)
knn = KNeighborsClassifier()
lr = LogisticRegression(random_state = SEED)
nb = GaussianNB()
dt = DecisionTreeClassifier(random_state = SEED)
rf = RandomForestClassifier(random_state = SEED)
extra = ExtraTreesClassifier(random_state = SEED)
boost = xgb.XGBClassifier(random_state = SEED)
mlp = MLPClassifier(random_state = SEED, max_iter = 1000)

#### 2.1 SVM

In [15]:
avalia_modelo(svm, dataset, y)

Modelo: SVC(random_state=912039)
Acurácia --> 0.69
Precisão --> 0.70
Recall --> 0.67
F1 --> 0.68
ROC AUC --> 0.69


In [16]:
avalia_modelo(svm, dataset_modificado, y)

Modelo: SVC(random_state=912039)
Acurácia --> 0.69
Precisão --> 0.70
Recall --> 0.67
F1 --> 0.68
ROC AUC --> 0.69


#### 2.2 KNN

In [17]:
avalia_modelo(knn, dataset, y)

Modelo: KNeighborsClassifier()
Acurácia --> 0.69
Precisão --> 0.69
Recall --> 0.68
F1 --> 0.68
ROC AUC --> 0.69


In [18]:
avalia_modelo(knn, dataset_modificado, y)

Modelo: KNeighborsClassifier()
Acurácia --> 0.69
Precisão --> 0.69
Recall --> 0.68
F1 --> 0.68
ROC AUC --> 0.69


#### 2.3 Árvore de Decisão

In [19]:
avalia_modelo(dt, dataset, y)

Modelo: DecisionTreeClassifier(random_state=912039)
Acurácia --> 0.67
Precisão --> 0.67
Recall --> 0.67
F1 --> 0.67
ROC AUC --> 0.67


In [20]:
avalia_modelo(dt, dataset_modificado, y)

Modelo: DecisionTreeClassifier(random_state=912039)
Acurácia --> 0.67
Precisão --> 0.67
Recall --> 0.67
F1 --> 0.67
ROC AUC --> 0.67


O uso de colunas adicionais não impactou no resultado modelo, como esperado, já que são valores constantes. Vamos abandonar isso.

#### 2.4 Bagging

In [21]:
avalia_modelo(rf, dataset, y)

Modelo: RandomForestClassifier(random_state=912039)
Acurácia --> 0.73
Precisão --> 0.74
Recall --> 0.71
F1 --> 0.73
ROC AUC --> 0.73


In [22]:
avalia_modelo(extra, dataset, y)

Modelo: ExtraTreesClassifier(random_state=912039)
Acurácia --> 0.71
Precisão --> 0.73
Recall --> 0.69
F1 --> 0.70
ROC AUC --> 0.71


#### 2.5 Naive Bayes

In [23]:
avalia_modelo(nb, dataset, y)

Modelo: GaussianNB()
Acurácia --> 0.53
Precisão --> 0.52
Recall --> 0.98
F1 --> 0.68
ROC AUC --> 0.53


#### 2.6 Regressão Logística

In [24]:
avalia_modelo(LogisticRegression(random_state = SEED, max_iter = 1000), dataset, y)

Modelo: LogisticRegression(max_iter=1000, random_state=912039)
Acurácia --> 0.70
Precisão --> 0.69
Recall --> 0.75
F1 --> 0.72
ROC AUC --> 0.70


#### 2.7 Boosting

In [25]:
avalia_modelo(boost, dataset, y)

Modelo: XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=912039, ...)
Acurácia --> 0.75
Precisão --> 0.75
Recall --> 0.75
F1 --> 0.75
ROC AUC --> 0.75


#### 2.8 MLP

In [26]:
avalia_modelo(mlp, dataset, y)

Modelo: MLPClassifier(max_iter=1000, random_state=912039)
Acurácia --> 0.74
Precisão --> 0.75
Recall --> 0.72
F1 --> 0.73
ROC AUC --> 0.74


Os resultados são bem parecidos. Todos os ensemble learners e a MLP apresentam um melhor desempenho, como esperado, e, fora isso, todos os outros classificadores ficam dentro de um limite próximo de desempenho, com exceção do Naive Bayes. Ele provavelmente está tendencioso para uma das classes.

No fim das contas, podemos ter os seguintes modelos em nosso Voting ou Stacking:

- Uma Máquina de Vetor de Suporte
- Uma Algoritmo de Vizinhos
- Uma Regressão Logística
- Um Naive Bayes
- Um Bagging
- Um Boosting
- Uma Rede Neural
- Talvez, uma Árvore de Decisão

Faremos o seguinte: __vamos otimizar cada um destes modelos, fazer um ranking, e testar a performance do Voting e do Stacking com 2, 3, 4 modelos... para determinar a melhor escolha__.

### 3. Otimizando Modelos

In [27]:
from sklearn.model_selection import RandomizedSearchCV

#### 3.1 SVM

In [28]:
"""
parametros = {
    'C': np.logspace(-3, 3, 7),
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'gamma': ['scale', 'auto'],
    'degree': [2, 3, 4],
    'shrinking': [True, False],
}

busca = RandomizedSearchCV(SVC(random_state = SEED), 
                           parametros, n_iter=30, cv=5, 
                           scoring='roc_auc', random_state=SEED, 
                           n_jobs=-1, verbose = 2)
busca.fit(dataset, y)

print(busca.best_estimator_)
print(busca.best_score_ * 100)
"""

# Omitido para melhorar execução, melhor modelo abaixo.

"\nparametros = {\n    'C': np.logspace(-3, 3, 7),\n    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],\n    'gamma': ['scale', 'auto'],\n    'degree': [2, 3, 4],\n    'shrinking': [True, False],\n}\n\nbusca = RandomizedSearchCV(SVC(random_state = SEED), \n                           parametros, n_iter=30, cv=5, \n                           scoring='roc_auc', random_state=SEED, \n                           n_jobs=-1, verbose = 2)\nbusca.fit(dataset, y)\n\nprint(busca.best_estimator_)\nprint(busca.best_score_ * 100)\n"

In [29]:
avalia_modelo(SVC(C=100.0, degree=2, gamma='auto', kernel='linear', random_state=912039), dataset, y)

Modelo: SVC(C=100.0, degree=2, gamma='auto', kernel='linear', random_state=912039)
Acurácia --> 0.70
Precisão --> 0.69
Recall --> 0.71
F1 --> 0.70
ROC AUC --> 0.70


In [30]:
MAQUINA_VETOR_SUPORTE = SVC(C=100.0, degree=2, gamma='auto', kernel='linear', random_state=912039, probability = True)

#### 3.2 KNN

In [31]:
"""
parametros = {
    'n_neighbors': np.arange(1, 21), 
    'weights': ['uniform', 'distance'],  
    'p': [1, 2], 
}

busca = RandomizedSearchCV(KNeighborsClassifier(), 
                           parametros, n_iter=30, cv=5, 
                           scoring='roc_auc', random_state=SEED, 
                           n_jobs=-1, verbose = 2)
busca.fit(dataset, y)

print(busca.best_estimator_)
print(busca.best_score_ * 100)
"""

"\nparametros = {\n    'n_neighbors': np.arange(1, 21), \n    'weights': ['uniform', 'distance'],  \n    'p': [1, 2], \n}\n\nbusca = RandomizedSearchCV(KNeighborsClassifier(), \n                           parametros, n_iter=30, cv=5, \n                           scoring='roc_auc', random_state=SEED, \n                           n_jobs=-1, verbose = 2)\nbusca.fit(dataset, y)\n\nprint(busca.best_estimator_)\nprint(busca.best_score_ * 100)\n"

In [32]:
avalia_modelo(KNeighborsClassifier(n_neighbors=19, p=1), dataset, y)

Modelo: KNeighborsClassifier(n_neighbors=19, p=1)
Acurácia --> 0.73
Precisão --> 0.75
Recall --> 0.68
F1 --> 0.72
ROC AUC --> 0.73


In [33]:
KNN = KNeighborsClassifier(n_neighbors=19, p=1)

#### 3.3 Árvore de Decisão

In [34]:
"""
parametros = {
    'criterion': ['gini', 'entropy'],  
    'splitter': ['best', 'random'],  
    'max_depth': [None] + list(np.arange(1, 21)),  
    'min_samples_split': list(np.arange(2, 21)),  
    'min_samples_leaf': list(np.arange(1, 21)), 
    'max_features': ['sqrt', 'log2', None],
    'max_leaf_nodes': [None] + list(np.arange(2, 21)) 
}

busca = RandomizedSearchCV(DecisionTreeClassifier(random_state = SEED), 
                           parametros, n_iter=100, cv=5, 
                           scoring='roc_auc', random_state=SEED, 
                           n_jobs=-1, verbose = 1)
busca.fit(dataset, y)

print(busca.best_estimator_)
print(busca.best_score_ * 100)
"""

"\nparametros = {\n    'criterion': ['gini', 'entropy'],  \n    'splitter': ['best', 'random'],  \n    'max_depth': [None] + list(np.arange(1, 21)),  \n    'min_samples_split': list(np.arange(2, 21)),  \n    'min_samples_leaf': list(np.arange(1, 21)), \n    'max_features': ['sqrt', 'log2', None],\n    'max_leaf_nodes': [None] + list(np.arange(2, 21)) \n}\n\nbusca = RandomizedSearchCV(DecisionTreeClassifier(random_state = SEED), \n                           parametros, n_iter=100, cv=5, \n                           scoring='roc_auc', random_state=SEED, \n                           n_jobs=-1, verbose = 1)\nbusca.fit(dataset, y)\n\nprint(busca.best_estimator_)\nprint(busca.best_score_ * 100)\n"

In [35]:
avalia_modelo(DecisionTreeClassifier(criterion='entropy', max_depth=13, max_leaf_nodes=12,
                                     min_samples_leaf=20, min_samples_split=7,random_state=912039), 
              dataset, y)

Modelo: DecisionTreeClassifier(criterion='entropy', max_depth=13, max_leaf_nodes=12,
                       min_samples_leaf=20, min_samples_split=7,
                       random_state=912039)
Acurácia --> 0.75
Precisão --> 0.77
Recall --> 0.72
F1 --> 0.74
ROC AUC --> 0.75


In [36]:
ARVORE_DECISAO = DecisionTreeClassifier(criterion='entropy', max_depth=13, max_leaf_nodes=12,
                                        min_samples_leaf=20, min_samples_split=7,random_state=912039)

#### 3.4 Regressão Logística

In [37]:
"""
parametros = {
    'C': np.logspace(-3, 3, 7),
    'solver': ['lbfgs', 'liblinear'], 
    'max_iter': np.arange(100, 1000, 100),
}

busca = RandomizedSearchCV(LogisticRegression(random_state = SEED), 
                           parametros, n_iter=30, cv=5, 
                           scoring='roc_auc', random_state=SEED, 
                           n_jobs=-1, verbose = 2)
busca.fit(dataset, y)

print(busca.best_estimator_)
print(busca.best_score_ * 100)
"""

"\nparametros = {\n    'C': np.logspace(-3, 3, 7),\n    'solver': ['lbfgs', 'liblinear'], \n    'max_iter': np.arange(100, 1000, 100),\n}\n\nbusca = RandomizedSearchCV(LogisticRegression(random_state = SEED), \n                           parametros, n_iter=30, cv=5, \n                           scoring='roc_auc', random_state=SEED, \n                           n_jobs=-1, verbose = 2)\nbusca.fit(dataset, y)\n\nprint(busca.best_estimator_)\nprint(busca.best_score_ * 100)\n"

In [38]:
avalia_modelo(LogisticRegression(C=10.0, max_iter=600, random_state=912039), dataset, y)

Modelo: LogisticRegression(C=10.0, max_iter=600, random_state=912039)
Acurácia --> 0.70
Precisão --> 0.69
Recall --> 0.75
F1 --> 0.72
ROC AUC --> 0.70


In [39]:
REGRESSAO_LOGISTICA = LogisticRegression(C=10.0, max_iter=600, random_state=912039)

#### 3.5 Bagging

In [40]:
"""
parametros = {
    'n_estimators': np.arange(50, 501, 50),
    'max_depth': [None] + list(np.arange(10, 101, 10)),
    'min_samples_split': np.arange(2, 11),
    'min_samples_leaf': np.arange(1, 11),
    'max_features': ['sqrt', 'log2', None]
}

busca = RandomizedSearchCV(RandomForestClassifier(random_state = SEED), 
                           parametros, n_iter=30, cv=5, 
                           scoring='roc_auc', random_state=SEED, 
                           n_jobs=-1, verbose = 2)
busca.fit(dataset, y)

print(busca.best_estimator_)
print(busca.best_score_ * 100)
"""

"\nparametros = {\n    'n_estimators': np.arange(50, 501, 50),\n    'max_depth': [None] + list(np.arange(10, 101, 10)),\n    'min_samples_split': np.arange(2, 11),\n    'min_samples_leaf': np.arange(1, 11),\n    'max_features': ['sqrt', 'log2', None]\n}\n\nbusca = RandomizedSearchCV(RandomForestClassifier(random_state = SEED), \n                           parametros, n_iter=30, cv=5, \n                           scoring='roc_auc', random_state=SEED, \n                           n_jobs=-1, verbose = 2)\nbusca.fit(dataset, y)\n\nprint(busca.best_estimator_)\nprint(busca.best_score_ * 100)\n"

In [41]:
avalia_modelo(RandomForestClassifier(max_depth=10, max_features=None, min_samples_leaf=2,
                                     min_samples_split=7, n_estimators=250,
                                     random_state=912039),
              dataset, y)

Modelo: RandomForestClassifier(max_depth=10, max_features=None, min_samples_leaf=2,
                       min_samples_split=7, n_estimators=250,
                       random_state=912039)
Acurácia --> 0.76
Precisão --> 0.76
Recall --> 0.76
F1 --> 0.76
ROC AUC --> 0.76


In [42]:
BAGGING = RandomForestClassifier(max_depth=10, max_features=None, min_samples_leaf=2,
                                     min_samples_split=7, n_estimators=250,
                                     random_state=912039)

#### 3.6 Boosting

In [43]:
"""
parametros = {
    'n_estimators': np.arange(50, 501, 50), 
    'max_depth': np.arange(3, 11), 
    'learning_rate': [0.001, 0.01, 0.1, 0.2, 0.3], 
    'subsample': [0.8, 0.9, 1.0], 
    'colsample_bytree': [0.8, 0.9, 1.0], 
}

busca = RandomizedSearchCV(xgb.XGBClassifier(random_state = SEED), 
                           parametros, n_iter=30, cv=5, 
                           scoring='roc_auc', random_state=SEED, 
                           n_jobs=-1, verbose = 2)
busca.fit(dataset, y)

print(busca.best_estimator_)
print(busca.best_score_ * 100)
"""

"\nparametros = {\n    'n_estimators': np.arange(50, 501, 50), \n    'max_depth': np.arange(3, 11), \n    'learning_rate': [0.001, 0.01, 0.1, 0.2, 0.3], \n    'subsample': [0.8, 0.9, 1.0], \n    'colsample_bytree': [0.8, 0.9, 1.0], \n}\n\nbusca = RandomizedSearchCV(xgb.XGBClassifier(random_state = SEED), \n                           parametros, n_iter=30, cv=5, \n                           scoring='roc_auc', random_state=SEED, \n                           n_jobs=-1, verbose = 2)\nbusca.fit(dataset, y)\n\nprint(busca.best_estimator_)\nprint(busca.best_score_ * 100)\n"

In [44]:
BOOSTING = xgb.XGBClassifier(random_state = SEED, n_estimators = 350, learning_rate = 0.01,
                             colsample_bytree = 0.9, max_depth = 5, subsample = 1.0)

In [45]:
avalia_modelo(BOOSTING, dataset, y)

Modelo: XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.9, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.01, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=350, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=912039, ...)
Acurácia --> 0.77
Precisão --> 0.77
Recall --> 0.76
F1 --> 0.77
ROC AUC --> 0.77


#### 3.7 MLP

In [46]:
"""
parametros = {
    'hidden_layer_sizes': [(50, 50), (100, 100), (50, 100, 50)],
    'activation': ['logistic', 'tanh', 'relu'], 
    'alpha': [0.0001, 0.001, 0.01], 
    'learning_rate_init': [0.001, 0.01, 0.1],
    'max_iter': [1000],
    'early_stopping': [True]
}

busca = RandomizedSearchCV(MLPClassifier(random_state = SEED), 
                           parametros, n_iter=30, cv=5, 
                           scoring='roc_auc', random_state=SEED, 
                           n_jobs=-1, verbose = 2)
busca.fit(dataset, y)

print(busca.best_estimator_)
print(busca.best_score_ * 100)
"""

"\nparametros = {\n    'hidden_layer_sizes': [(50, 50), (100, 100), (50, 100, 50)],\n    'activation': ['logistic', 'tanh', 'relu'], \n    'alpha': [0.0001, 0.001, 0.01], \n    'learning_rate_init': [0.001, 0.01, 0.1],\n    'max_iter': [1000],\n    'early_stopping': [True]\n}\n\nbusca = RandomizedSearchCV(MLPClassifier(random_state = SEED), \n                           parametros, n_iter=30, cv=5, \n                           scoring='roc_auc', random_state=SEED, \n                           n_jobs=-1, verbose = 2)\nbusca.fit(dataset, y)\n\nprint(busca.best_estimator_)\nprint(busca.best_score_ * 100)\n"

In [47]:
NEURAL = MLPClassifier(activation='tanh', alpha=0.001, early_stopping=True,
                       hidden_layer_sizes=(50, 50), learning_rate_init=0.01,
                       max_iter=1000, random_state=912039)

In [48]:
avalia_modelo(NEURAL, dataset, y)

Modelo: MLPClassifier(activation='tanh', alpha=0.001, early_stopping=True,
              hidden_layer_sizes=(50, 50), learning_rate_init=0.01,
              max_iter=1000, random_state=912039)
Acurácia --> 0.73
Precisão --> 0.73
Recall --> 0.74
F1 --> 0.73
ROC AUC --> 0.73


O modelo original teve melhor performance.

In [49]:
NEURAL = MLPClassifier(max_iter = 1000, random_state = SEED)

### 4. Resultados

Com os modelos otimizados, o resultado foi o ranking abaixo. Agora, vamos usar isso de parâmetro para a criação dos classificadores conjuntos.


1. XGBoost - 0.77 (BOOSTING)
2. RandomForest - 0.76 (BAGGING)
3. Árvore de Decisão - 0.75 (ARVORE_DECISAO)
4. Rede Neural - 0.74 (NEURAL)
5. KNN - 0.73 (KNN)
6. SVM - 0.70 (MAQUINA_VETOR_SUPORTE), Regressão Logística - 0.70 (REGRESSAO_LOGISTICA)
7. Naive Bayes - 0.53 (Com 98% de Recall)

### 5. VotingClassifier

In [50]:
from sklearn.ensemble import VotingClassifier, StackingClassifier

#### 3 Classificadores

In [51]:
"""
classificador = VotingClassifier(estimators=[
                                            ('Xgboost', BOOSTING),
                                            ('RandomForest', BAGGING),
                                            ('Árvore de Decisão', ARVORE_DECISAO)
                                        ], voting='soft')

avalia_modelo(classificador, dataset, y)
"""

"\nclassificador = VotingClassifier(estimators=[\n                                            ('Xgboost', BOOSTING),\n                                            ('RandomForest', BAGGING),\n                                            ('Árvore de Decisão', ARVORE_DECISAO)\n                                        ], voting='soft')\n\navalia_modelo(classificador, dataset, y)\n"

#### 4 Classificadores

In [52]:
"""
classificador = VotingClassifier(estimators=[
                                            ('Xgboost', BOOSTING),
                                            ('RandomForest', BAGGING),
                                            ('Árvore de Decisão', ARVORE_DECISAO),
                                            ('Rede Neural', NEURAL)
                                        ], voting='soft')

avalia_modelo(classificador, dataset, y)
"""

"\nclassificador = VotingClassifier(estimators=[\n                                            ('Xgboost', BOOSTING),\n                                            ('RandomForest', BAGGING),\n                                            ('Árvore de Decisão', ARVORE_DECISAO),\n                                            ('Rede Neural', NEURAL)\n                                        ], voting='soft')\n\navalia_modelo(classificador, dataset, y)\n"

#### 5 Classificadores

In [53]:
"""
classificador = VotingClassifier(estimators=[
                                            ('Xgboost', BOOSTING),
                                            ('RandomForest', BAGGING),
                                            ('Árvore de Decisão', ARVORE_DECISAO),
                                            ('Rede Neural', NEURAL),
                                            ('KNN', KNN)
                                        ], voting='soft')

avalia_modelo(classificador, dataset, y)
"""

"\nclassificador = VotingClassifier(estimators=[\n                                            ('Xgboost', BOOSTING),\n                                            ('RandomForest', BAGGING),\n                                            ('Árvore de Decisão', ARVORE_DECISAO),\n                                            ('Rede Neural', NEURAL),\n                                            ('KNN', KNN)\n                                        ], voting='soft')\n\navalia_modelo(classificador, dataset, y)\n"

#### 6 Classificadores

In [54]:
"""
classificador = VotingClassifier(estimators=[
                                            ('Xgboost', BOOSTING),
                                            ('RandomForest', BAGGING),
                                            ('Árvore de Decisão', ARVORE_DECISAO),
                                            ('Rede Neural', NEURAL),
                                            ('KNN', KNN),
                                            ('Máquina de Vetor de Suporte', MAQUINA_VETOR_SUPORTE)
                                        ], voting='soft')

avalia_modelo(classificador, dataset, y)
"""

"\nclassificador = VotingClassifier(estimators=[\n                                            ('Xgboost', BOOSTING),\n                                            ('RandomForest', BAGGING),\n                                            ('Árvore de Decisão', ARVORE_DECISAO),\n                                            ('Rede Neural', NEURAL),\n                                            ('KNN', KNN),\n                                            ('Máquina de Vetor de Suporte', MAQUINA_VETOR_SUPORTE)\n                                        ], voting='soft')\n\navalia_modelo(classificador, dataset, y)\n"

#### 7 Classificadores

In [55]:
"""
classificador = VotingClassifier(estimators=[
                                            ('Xgboost', BOOSTING),
                                            ('RandomForest', BAGGING),
                                            ('Árvore de Decisão', ARVORE_DECISAO),
                                            ('Rede Neural', NEURAL),
                                            ('KNN', KNN),
                                            ('Máquina de Vetor de Suporte', MAQUINA_VETOR_SUPORTE),
                                            ('Regressão Logística', REGRESSAO_LOGISTICA)
                                        ], voting='soft')

avalia_modelo(classificador, dataset, y)
"""

"\nclassificador = VotingClassifier(estimators=[\n                                            ('Xgboost', BOOSTING),\n                                            ('RandomForest', BAGGING),\n                                            ('Árvore de Decisão', ARVORE_DECISAO),\n                                            ('Rede Neural', NEURAL),\n                                            ('KNN', KNN),\n                                            ('Máquina de Vetor de Suporte', MAQUINA_VETOR_SUPORTE),\n                                            ('Regressão Logística', REGRESSAO_LOGISTICA)\n                                        ], voting='soft')\n\navalia_modelo(classificador, dataset, y)\n"

#### 8 Classificadores

In [56]:
"""
classificador = VotingClassifier(estimators=[
                                            ('Xgboost', BOOSTING),
                                            ('RandomForest', BAGGING),
                                            ('Árvore de Decisão', ARVORE_DECISAO),
                                            ('Rede Neural', NEURAL),
                                            ('KNN', KNN),
                                            ('Máquina de Vetor de Suporte', MAQUINA_VETOR_SUPORTE),
                                            ('Regressão Logística', REGRESSAO_LOGISTICA),
                                            ('Bayes', GaussianNB())
                                        ], voting='soft')

avalia_modelo(classificador, dataset, y)
"""

"\nclassificador = VotingClassifier(estimators=[\n                                            ('Xgboost', BOOSTING),\n                                            ('RandomForest', BAGGING),\n                                            ('Árvore de Decisão', ARVORE_DECISAO),\n                                            ('Rede Neural', NEURAL),\n                                            ('KNN', KNN),\n                                            ('Máquina de Vetor de Suporte', MAQUINA_VETOR_SUPORTE),\n                                            ('Regressão Logística', REGRESSAO_LOGISTICA),\n                                            ('Bayes', GaussianNB())\n                                        ], voting='soft')\n\navalia_modelo(classificador, dataset, y)\n"

### 6. Resultados

In [57]:
alvo = pd.read_csv("/kaggle/input/santander-customer-satisfaction/test.csv")

alvo_id = alvo.ID
alvo.drop(columns = "ID", inplace = True)

scaler = MinMaxScaler()

copia = alvo.copy()
copia = scaler.fit_transform(copia)

alvo = pd.DataFrame(copia, columns = alvo.columns)
alvo.head()

,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,imp_op_var40_ult1,...,saldo_medio_var29_ult3,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38
0,0.999764,0.27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000004,0.000004,0.0,0.0,0.0,0.0,0.001361
1,0.999764,0.30,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000004,0.000004,0.0,0.0,0.0,0.0,0.001533
2,0.999764,0.18,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000004,0.000004,0.0,0.0,0.0,0.0,0.001585
3,0.999764,0.19,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000004,0.000004,0.0,0.0,0.0,0.0,0.006462
4,0.999764,0.18,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000004,0.000004,0.0,0.0,0.0,0.0,0.002507


In [58]:
alvo = alvo.drop(columns = COLUNAS_CONSTANTES)
alvo.head()

,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,imp_op_var40_ult1,...,saldo_medio_var29_ult3,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38
0,0.999764,0.27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000004,0.000004,0.0,0.0,0.0,0.0,0.001361
1,0.999764,0.30,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000004,0.000004,0.0,0.0,0.0,0.0,0.001533
2,0.999764,0.18,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000004,0.000004,0.0,0.0,0.0,0.0,0.001585
3,0.999764,0.19,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000004,0.000004,0.0,0.0,0.0,0.0,0.006462
4,0.999764,0.18,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000004,0.000004,0.0,0.0,0.0,0.0,0.002507


#### Voting e Stacking com 3 Classificadores

In [59]:
"""
classificador = VotingClassifier(estimators=[
                                            ('Xgboost', BOOSTING),
                                            ('RandomForest', BAGGING),
                                            ('Árvore de Decisão', ARVORE_DECISAO)
                                        ], voting='soft')
classificador.fit(dataset_modificado, y)
"""

"\nclassificador = VotingClassifier(estimators=[\n                                            ('Xgboost', BOOSTING),\n                                            ('RandomForest', BAGGING),\n                                            ('Árvore de Decisão', ARVORE_DECISAO)\n                                        ], voting='soft')\nclassificador.fit(dataset_modificado, y)\n"

In [60]:
"""
resultados = classificador.predict(alvo)
resposta = pd.concat([alvo_id, pd.Series(resultados, name = "TARGET")], axis = 1)
resposta
"""

'\nresultados = classificador.predict(alvo)\nresposta = pd.concat([alvo_id, pd.Series(resultados, name = "TARGET")], axis = 1)\nresposta\n'

In [61]:
"""resposta.to_csv("voting_3_classificadores.csv", index = False)"""

'resposta.to_csv("voting_3_classificadores.csv", index = False)'

In [103]:
"""
classificador = StackingClassifier(estimators=[
                                            ('Xgboost', BOOSTING),
                                            ('RandomForest', BAGGING),
                                            ('Árvore de Decisão', ARVORE_DECISAO)
                                        ], final_estimator = LogisticRegression(random_state = SEED))
classificador.fit(dataset_modificado, y)
"""

StackingClassifier(estimators=[('Xgboost',
                                XGBClassifier(base_score=None, booster=None,
                                              callbacks=None,
                                              colsample_bylevel=None,
                                              colsample_bynode=None,
                                              colsample_bytree=0.9,
                                              early_stopping_rounds=None,
                                              enable_categorical=False,
                                              eval_metric=None,
                                              feature_types=None, gamma=None,
                                              gpu_id=None, grow_policy=None,
                                              importance_type=None,
                                              interaction_constraints=None,
                                              learni...
                               ('RandomForest',
                                RandomForestClassifier(max_depth=10,
                                                       max_features=None,
                                                       min_samples_leaf=2,
                                                       min_samples_split=7,
                                                       n_estimators=250,
                                                       random_state=912039)),
                               ('Árvore de Decisão',
                                DecisionTreeClassifier(criterion='entropy',
                                                       max_depth=13,
                                                       max_leaf_nodes=12,
                                                       min_samples_leaf=20,
                                                       min_samples_split=7,
                                                       random_state=912039))],
                   final_estimator=LogisticRegression(random_state=912039))

In [104]:
"""
resultados = classificador.predict(alvo)
resposta = pd.concat([alvo_id, pd.Series(resultados, name = "TARGET")], axis = 1)
resposta
"""

,ID,TARGET
0,2,1
1,5,1
2,6,0
3,7,0
4,9,0
...,...,...
75813,151831,0
75814,151832,0
75815,151833,0
75816,151834,1


In [105]:
"""resposta.to_csv("stacking_3_classificadores.csv", index = False)"""

#### Voting com 4 Classificadores

In [62]:
"""
classificador = VotingClassifier(estimators=[
                                            ('Xgboost', BOOSTING),
                                            ('RandomForest', BAGGING),
                                            ('Árvore de Decisão', ARVORE_DECISAO),
                                            ('Rede Neural', NEURAL)
                                        ], voting='soft')
classificador.fit(dataset_modificado, y)
"""

"\nclassificador = VotingClassifier(estimators=[\n                                            ('Xgboost', BOOSTING),\n                                            ('RandomForest', BAGGING),\n                                            ('Árvore de Decisão', ARVORE_DECISAO),\n                                            ('Rede Neural', NEURAL)\n                                        ], voting='soft')\nclassificador.fit(dataset_modificado, y)\n"

In [63]:
"""
resultados = classificador.predict(alvo)
resposta = pd.concat([alvo_id, pd.Series(resultados, name = "TARGET")], axis = 1)
resposta
"""

'\nresultados = classificador.predict(alvo)\nresposta = pd.concat([alvo_id, pd.Series(resultados, name = "TARGET")], axis = 1)\nresposta\n'

In [64]:
"""resposta.to_csv("voting_4_classificadores.csv", index = False)"""

'resposta.to_csv("voting_4_classificadores.csv", index = False)'

#### Voting com 5 Classificadores

In [65]:
"""
classificador = VotingClassifier(estimators=[
                                            ('Xgboost', BOOSTING),
                                            ('RandomForest', BAGGING),
                                            ('Árvore de Decisão', ARVORE_DECISAO),
                                            ('Rede Neural', NEURAL),
                                            ('KNN', KNN)
                                        ], voting='soft')
classificador.fit(dataset_modificado, y)
"""

"\nclassificador = VotingClassifier(estimators=[\n                                            ('Xgboost', BOOSTING),\n                                            ('RandomForest', BAGGING),\n                                            ('Árvore de Decisão', ARVORE_DECISAO),\n                                            ('Rede Neural', NEURAL),\n                                            ('KNN', KNN)\n                                        ], voting='soft')\nclassificador.fit(dataset_modificado, y)\n"

In [66]:
"""
resultados = classificador.predict(alvo)
resposta = pd.concat([alvo_id, pd.Series(resultados, name = "TARGET")], axis = 1)
resposta
"""

'\nresultados = classificador.predict(alvo)\nresposta = pd.concat([alvo_id, pd.Series(resultados, name = "TARGET")], axis = 1)\nresposta\n'

In [67]:
"""resposta.to_csv("voting_5_classificadores.csv", index = False)"""

'resposta.to_csv("voting_5_classificadores.csv", index = False)'

#### Voting com 6 Classificadores

In [68]:
"""
classificador = VotingClassifier(estimators=[
                                            ('Xgboost', BOOSTING),
                                            ('RandomForest', BAGGING),
                                            ('Árvore de Decisão', ARVORE_DECISAO),
                                            ('Rede Neural', NEURAL),
                                            ('KNN', KNN),
                                            ('Máquina de Vetor de Suporte', MAQUINA_VETOR_SUPORTE)
                                        ], voting='soft')
classificador.fit(dataset_modificado, y)
"""

"\nclassificador = VotingClassifier(estimators=[\n                                            ('Xgboost', BOOSTING),\n                                            ('RandomForest', BAGGING),\n                                            ('Árvore de Decisão', ARVORE_DECISAO),\n                                            ('Rede Neural', NEURAL),\n                                            ('KNN', KNN),\n                                            ('Máquina de Vetor de Suporte', MAQUINA_VETOR_SUPORTE)\n                                        ], voting='soft')\nclassificador.fit(dataset_modificado, y)\n"

In [69]:
"""
resultados = classificador.predict(alvo)
resposta = pd.concat([alvo_id, pd.Series(resultados, name = "TARGET")], axis = 1)
resposta
"""

'\nresultados = classificador.predict(alvo)\nresposta = pd.concat([alvo_id, pd.Series(resultados, name = "TARGET")], axis = 1)\nresposta\n'

In [70]:
"""resposta.to_csv("voting_6_classificadores.csv", index = False)"""

'resposta.to_csv("voting_6_classificadores.csv", index = False)'

#### Voting com 7 Classificadores

In [71]:
"""
classificador = VotingClassifier(estimators=[
                                            ('Xgboost', BOOSTING),
                                            ('RandomForest', BAGGING),
                                            ('Árvore de Decisão', ARVORE_DECISAO),
                                            ('Rede Neural', NEURAL),
                                            ('KNN', KNN),
                                            ('Máquina de Vetor de Suporte', MAQUINA_VETOR_SUPORTE),
                                            ('Regressão Logística', REGRESSAO_LOGISTICA)
                                        ], voting='soft')
classificador.fit(dataset_modificado, y)
"""

"\nclassificador = VotingClassifier(estimators=[\n                                            ('Xgboost', BOOSTING),\n                                            ('RandomForest', BAGGING),\n                                            ('Árvore de Decisão', ARVORE_DECISAO),\n                                            ('Rede Neural', NEURAL),\n                                            ('KNN', KNN),\n                                            ('Máquina de Vetor de Suporte', MAQUINA_VETOR_SUPORTE),\n                                            ('Regressão Logística', REGRESSAO_LOGISTICA)\n                                        ], voting='soft')\nclassificador.fit(dataset_modificado, y)\n"

In [72]:
"""
resultados = classificador.predict(alvo)
resposta = pd.concat([alvo_id, pd.Series(resultados, name = "TARGET")], axis = 1)
resposta
"""

'\nresultados = classificador.predict(alvo)\nresposta = pd.concat([alvo_id, pd.Series(resultados, name = "TARGET")], axis = 1)\nresposta\n'

In [73]:
"""resposta.to_csv("voting_7_classificadores.csv", index = False)"""

'resposta.to_csv("voting_7_classificadores.csv", index = False)'

In [113]:
"""
classificador = StackingClassifier(estimators=[
                                            ('Xgboost', BOOSTING),
                                            ('RandomForest', BAGGING),
                                            ('Árvore de Decisão', ARVORE_DECISAO),
                                            ('Rede Neural', NEURAL),
                                            ('KNN', KNN),
                                            ('Máquina de Vetor de Suporte', MAQUINA_VETOR_SUPORTE),
                                            ('Regressão Logística', REGRESSAO_LOGISTICA),
                                        ], final_estimator = LogisticRegression(random_state = SEED))
classificador.fit(dataset_modificado, y)
"""

StackingClassifier(estimators=[('Xgboost',
                                XGBClassifier(base_score=None, booster=None,
                                              callbacks=None,
                                              colsample_bylevel=None,
                                              colsample_bynode=None,
                                              colsample_bytree=0.9,
                                              early_stopping_rounds=None,
                                              enable_categorical=False,
                                              eval_metric=None,
                                              feature_types=None, gamma=None,
                                              gpu_id=None, grow_policy=None,
                                              importance_type=None,
                                              interaction_constraints=None,
                                              learni...
                                MLPClassifier(max_iter=1000,
                                              random_state=912039)),
                               ('KNN',
                                KNeighborsClassifier(n_neighbors=19, p=1)),
                               ('Máquina de Vetor de Suporte',
                                SVC(C=100.0, degree=2, gamma='auto',
                                    kernel='linear', probability=True,
                                    random_state=912039)),
                               ('Regressão Logística',
                                LogisticRegression(C=10.0, max_iter=600,
                                                   random_state=912039))],
                   final_estimator=LogisticRegression(random_state=912039))

In [114]:
"""
resultados = classificador.predict(alvo)
resposta = pd.concat([alvo_id, pd.Series(resultados, name = "TARGET")], axis = 1)
resposta
"""

,ID,TARGET
0,2,1
1,5,1
2,6,0
3,7,0
4,9,0
...,...,...
75813,151831,0
75814,151832,0
75815,151833,0
75816,151834,1


In [115]:
"""resposta.to_csv("stacking_7_classificadores.csv", index = False)"""

#### Voting com 8 Classificadores

In [74]:
"""
classificador = VotingClassifier(estimators=[
                                            ('Xgboost', BOOSTING),
                                            ('RandomForest', BAGGING),
                                            ('Árvore de Decisão', ARVORE_DECISAO),
                                            ('Rede Neural', NEURAL),
                                            ('KNN', KNN),
                                            ('Máquina de Vetor de Suporte', MAQUINA_VETOR_SUPORTE),
                                            ('Regressão Logística', REGRESSAO_LOGISTICA),
                                            ('Bayes', GaussianNB())
                                        ], voting='soft')
classificador.fit(dataset_modificado, y)
"""

"\nclassificador = VotingClassifier(estimators=[\n                                            ('Xgboost', BOOSTING),\n                                            ('RandomForest', BAGGING),\n                                            ('Árvore de Decisão', ARVORE_DECISAO),\n                                            ('Rede Neural', NEURAL),\n                                            ('KNN', KNN),\n                                            ('Máquina de Vetor de Suporte', MAQUINA_VETOR_SUPORTE),\n                                            ('Regressão Logística', REGRESSAO_LOGISTICA),\n                                            ('Bayes', GaussianNB())\n                                        ], voting='soft')\nclassificador.fit(dataset_modificado, y)\n"

In [75]:
"""
resultados = classificador.predict(alvo)
resposta = pd.concat([alvo_id, pd.Series(resultados, name = "TARGET")], axis = 1)
resposta
"""

'\nresultados = classificador.predict(alvo)\nresposta = pd.concat([alvo_id, pd.Series(resultados, name = "TARGET")], axis = 1)\nresposta\n'

In [76]:
"""resposta.to_csv("voting_8_classificadores.csv", index = False)"""

'resposta.to_csv("voting_8_classificadores.csv", index = False)'

#### Recriando o VotingClassifier da 3ª Tentativa

In [77]:
"""
classificador = VotingClassifier(estimators=[
                                            ('Xgboost', BOOSTING),
                                            ('RandomForest', BAGGING),
                                            ('Rede Neural', NEURAL),
                                            ('KNN', KNN),
                                            ('Regressão Logística', REGRESSAO_LOGISTICA),
                                        ], voting='soft')
classificador.fit(dataset_modificado, y)
"""

VotingClassifier(estimators=[('Xgboost',
                              XGBClassifier(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=0.9,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric=None,
                                            feature_types=None, gamma=None,
                                            gpu_id=None, grow_policy=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning...
                             ('RandomForest',
                              RandomForestClassifier(max_depth=10,
                                                     max_features=None,
                                                     min_samples_leaf=2,
                                                     min_samples_split=7,
                                                     n_estimators=250,
                                                     random_state=912039)),
                             ('Rede Neural',
                              MLPClassifier(max_iter=1000,
                                            random_state=912039)),
                             ('KNN', KNeighborsClassifier(n_neighbors=19, p=1)),
                             ('Regressão Logística',
                              LogisticRegression(C=10.0, max_iter=600,
                                                 random_state=912039))],
                 voting='soft')

In [78]:
"""
resultados = classificador.predict(alvo)
resposta = pd.concat([alvo_id, pd.Series(resultados, name = "TARGET")], axis = 1)
resposta
"""

,ID,TARGET
0,2,1
1,5,1
2,6,0
3,7,0
4,9,0
...,...,...
75813,151831,0
75814,151832,0
75815,151833,0
75816,151834,0


In [79]:
"""resposta.to_csv("voting_3thAttempt_parameter.csv", index = False)"""

In [80]:
"""
classificador = VotingClassifier(estimators=[
                                            ('Xgboost', xgb.XGBClassifier(random_state = SEED)),
                                            ('RandomForest', RandomForestClassifier(random_state = SEED)),
                                            ('Rede Neural', MLPClassifier(random_state = SEED, max_iter = 1000)),
                                            ('KNN', KNeighborsClassifier()),
                                            ('Regressão Logística', LogisticRegression(random_state = SEED, max_iter = 1000)),
                                        ], voting='soft')
classificador.fit(dataset_modificado, y)
"""

VotingClassifier(estimators=[('Xgboost',
                              XGBClassifier(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric=None,
                                            feature_types=None, gamma=None,
                                            gpu_id=None, grow_policy=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learnin...
                                            monotone_constraints=None,
                                            n_estimators=100, n_jobs=None,
                                            num_parallel_tree=None,
                                            predictor=None, random_state=912039, ...)),
                             ('RandomForest',
                              RandomForestClassifier(random_state=912039)),
                             ('Rede Neural',
                              MLPClassifier(max_iter=1000,
                                            random_state=912039)),
                             ('KNN', KNeighborsClassifier()),
                             ('Regressão Logística',
                              LogisticRegression(max_iter=1000,
                                                 random_state=912039))],
                 voting='soft')

In [81]:
"""
resultados = classificador.predict(alvo)
resposta = pd.concat([alvo_id, pd.Series(resultados, name = "TARGET")], axis = 1)
resposta
"""

,ID,TARGET
0,2,1
1,5,1
2,6,0
3,7,0
4,9,0
...,...,...
75813,151831,0
75814,151832,0
75815,151833,0
75816,151834,0


In [82]:
"""resposta.to_csv("voting_3thAttempt.csv", index = False)"""

In [109]:
"""
classificador = StackingClassifier(estimators=[
                                            ('Xgboost', xgb.XGBClassifier(random_state = SEED)),
                                            ('RandomForest', RandomForestClassifier(random_state = SEED)),
                                            ('Rede Neural', MLPClassifier(random_state = SEED, max_iter = 1000)),
                                            ('KNN', KNeighborsClassifier()),
                                            ('Regressão Logística', LogisticRegression(random_state = SEED, max_iter = 1000)),
                                        ], final_estimator = LogisticRegression(random_state = SEED))
classificador.fit(dataset_modificado, y)
"""


StackingClassifier(estimators=[('Xgboost',
                                XGBClassifier(base_score=None, booster=None,
                                              callbacks=None,
                                              colsample_bylevel=None,
                                              colsample_bynode=None,
                                              colsample_bytree=None,
                                              early_stopping_rounds=None,
                                              enable_categorical=False,
                                              eval_metric=None,
                                              feature_types=None, gamma=None,
                                              gpu_id=None, grow_policy=None,
                                              importance_type=None,
                                              interaction_constraints=None,
                                              learn...
                                              num_parallel_tree=None,
                                              predictor=None,
                                              random_state=912039, ...)),
                               ('RandomForest',
                                RandomForestClassifier(random_state=912039)),
                               ('Rede Neural',
                                MLPClassifier(max_iter=1000,
                                              random_state=912039)),
                               ('KNN', KNeighborsClassifier()),
                               ('Regressão Logística',
                                LogisticRegression(max_iter=1000,
                                                   random_state=912039))],
                   final_estimator=LogisticRegression(random_state=912039))

In [110]:
"""
resultados = classificador.predict(alvo)
resposta = pd.concat([alvo_id, pd.Series(resultados, name = "TARGET")], axis = 1)
resposta
"""

,ID,TARGET
0,2,1
1,5,1
2,6,0
3,7,0
4,9,0
...,...,...
75813,151831,0
75814,151832,0
75815,151833,0
75816,151834,0


In [111]:
"""resposta.to_csv("stacking_3thAttempt_logistic.csv", index = False)"""

#### Último Voting Classifier

In [97]:
"""
classificador = VotingClassifier(estimators=[
                                            ('Xgboost', xgb.XGBClassifier(random_state = SEED)),
                                            ('RandomForest', RandomForestClassifier(random_state = SEED)),
                                            ('Rede Neural', MLPClassifier(random_state = SEED, max_iter = 1000)),
                                            ('KNN', KNeighborsClassifier()),
                                            ('Regressão Logística', LogisticRegression(random_state = SEED, max_iter = 1000)),
                                            ('Máquina de Vetor', SVC(random_state = SEED, probability = True))
                                        ], voting='soft')
classificador.fit(dataset_modificado, y)
"""

VotingClassifier(estimators=[('Xgboost',
                              XGBClassifier(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric=None,
                                            feature_types=None, gamma=None,
                                            gpu_id=None, grow_policy=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learnin...
                                            predictor=None, random_state=912039, ...)),
                             ('RandomForest',
                              RandomForestClassifier(random_state=912039)),
                             ('Rede Neural',
                              MLPClassifier(max_iter=1000,
                                            random_state=912039)),
                             ('KNN', KNeighborsClassifier()),
                             ('Regressão Logística',
                              LogisticRegression(max_iter=1000,
                                                 random_state=912039)),
                             ('Máquina de Vetor',
                              SVC(probability=True, random_state=912039))],
                 voting='soft')

In [98]:
"""
resultados = classificador.predict(alvo)
resposta = pd.concat([alvo_id, pd.Series(resultados, name = "TARGET")], axis = 1)
resposta
"""

,ID,TARGET
0,2,1
1,5,1
2,6,0
3,7,0
4,9,0
...,...,...
75813,151831,0
75814,151832,0
75815,151833,0
75816,151834,0


In [99]:
"""resposta.to_csv("last_voting.csv", index = False)"""